In [1]:
import numpy as np
import pandas as pd
import glob

In [2]:
from numpy.fft import fft
import detecta
def compute_peaks(data):
    X=fft(data)
    N=len(X)
    n=np.arange(N)
    sr=1/50
    T=N/sr
    freq=n/T
    c=20

    n_oneside=N//2
    x=freq[1:n_oneside]
    y=np.abs(X[1:n_oneside])

    max_peak_height=np.amax(y)/c
    peaks=[]
    while len(peaks)<5:
        peaks=detecta.detect_peaks(y, mph=max_peak_height)
        c+=5
        max_peak_height=np.amax(y)/c
    peaks_x=peaks/T
    peaks_y=y[peaks]

    return peaks_x[0:5], peaks_y[0:5]

In [3]:
def find_fft_points(data, name):
    print(name)
    (indices_peaks, peaks) = compute_peaks(data)
    columns_x=[name + "X#1", name + "X#2", name + "X#3", name + "X#4", name + "X#5"]
    columns_y=[name + "P#1", name + "P#2", name + "P#3", name + "P#4", name + "P#5"]
    x_p = pd.DataFrame(data=indices_peaks).T
    x_p.columns = columns_x
    y_p = pd.DataFrame(data=peaks).T
    y_p.columns = columns_y
    tot_p = pd.concat([x_p, y_p], axis=1)
    return tot_p

In [4]:
def compute_time_features(df):
    data_total = pd.DataFrame()
    for column in df.columns:
        temp = find_time_features(df[column], column)
        data_total = pd.concat([data_total, temp], axis=1)
    return data_total

In [5]:
def find_time_features(data, name):
    columns = [name + "_mean", name + "_std", name + "_range", name + "_IRQ"]
    properties = [np.mean(data), np.std(data), np.max(data) - np.min(data),
                  np.quantile(data, 0.75) - np.quantile(data, 0.25)]
    d = pd.DataFrame(data=properties).T
    d.columns = columns
    return d

In [6]:
from scipy.signal import savgol_filter

def noise_filter(dataframe):
    for column in dataframe.columns:
        dataframe[column] = savgol_filter(dataframe[column], 10, 2, axis=0)
    return dataframe

In [7]:
def compute_freq_features(df):
    data_total = pd.DataFrame()
    for column in df.columns:
        temp = find_fft_points(df[column], column)
        data_total = pd.concat([data_total, temp], axis=1)
    return data_total

In [8]:
def collapse(df):
    time_df=compute_time_features(df)
    freq_df=compute_freq_features(df)
    return pd.concat([time_df, freq_df], axis=1)

In [9]:
def transform_to_timeseries(dataframe, class_name, sample_number):
    i = dataframe.shape[0]//sample_number
    j=0
    filtered_df=noise_filter(dataframe)
    df_time_series=pd.DataFrame()
    for count in range(1,i):
        print(j)
        print(sample_number*count)
        samples_df=filtered_df.iloc[j:sample_number*count, :]
        new_df=collapse(samples_df)
        if(count==1):
            df_time_series=new_df
        else:
            df_time_series = pd.concat([df_time_series, new_df], axis=0)
        j=sample_number*count
    df_time_series['class']=class_name
    return df_time_series

In [10]:
all_data=[]

i = 0
all_files=glob.glob("honor20readings/*", recursive=True)
for file in all_files:
    i = i + 1
    print(file)
    df=pd.read_csv(file)
    class_name=df.iloc[0]["class"]
    df=df.drop("class", axis=1)
    # Drop not - measurable columns
    temp_df = transform_to_timeseries(df, class_name, 150)
    all_data.append(temp_df)

df=pd.concat(all_data, axis=0, ignore_index=True)
df.to_csv("dataset_peaks_honor_3sec.csv")

honor20readings\dataset_honor20_downstairs.csv
0
150
rotationRate.x
rotationRate.y
rotationRate.z
userAcceleration.x
userAcceleration.y
userAcceleration.z
honor20readings\dataset_honor20_downstairs1.csv
honor20readings\dataset_honor20_downstairs2.csv
honor20readings\dataset_honor20_downstairs3.csv
honor20readings\dataset_honor20_jogging.csv
0
150
rotationRate.x
rotationRate.y
rotationRate.z
userAcceleration.x
userAcceleration.y
userAcceleration.z
150
300
rotationRate.x
rotationRate.y
rotationRate.z
userAcceleration.x
userAcceleration.y
userAcceleration.z
300
450
rotationRate.x
rotationRate.y
rotationRate.z
userAcceleration.x
userAcceleration.y
userAcceleration.z
450
600
rotationRate.x
rotationRate.y
rotationRate.z
userAcceleration.x
userAcceleration.y
userAcceleration.z
600
750
rotationRate.x
rotationRate.y
rotationRate.z
userAcceleration.x
userAcceleration.y
userAcceleration.z
750
900
rotationRate.x
rotationRate.y
rotationRate.z
userAcceleration.x
userAcceleration.y
userAcceleration.z